Andrea Palladio - Villa Badoer
================

Working again on Villa Badoer as started in workshop_05.
Aim of this final workshop is to completing the model.

In [2]:
from larlib import *
from math import floor
import csv
import os

Evaluating fenvs.py..
...fenvs.py imported in 0.011585 seconds


# Reading data
As mentioned in workshop_05 I will assign 3d-points instead of getting data from the filesystem. This will prevent problems regarding paths etc. this approach may come in handy for fine tune points ;)

In [3]:
def larModel(path2lines):
    """
    larModel("mypath")
    :param path2file: path to the .lines file
    :return: tuple. A graph model representing Vertices and Edges.
             Edges are lists of indices to Vertices.
    """
    with open(path2lines, "rb") as file:
        lines = csv.reader(file, delimiter=",")

        mapping = {}
        v = []
        ev = []
        index = 0
        
        # indexing of the vertices and approximation
        for entry in lines:
            x1 = math.floor(float(entry[0])*100)/100
            y1 = math.floor(float(entry[1])*100)/100
            x2 = math.floor(float(entry[2])*100)/100
            y2 = math.floor(float(entry[3])*100)/100

            if not(mapping.has_key((x1,y1))):
                index += 1
                mapping[(x1,y1)] = index
                v.append([x1,y1])

            if not(mapping.has_key((x2,y2))):
                index += 1
                mapping[(x2,y2)] = index
                v.append([x2,y2])

            ev.append([mapping[(x1,y1)], mapping[(x2,y2)]])

    return v,ev

# Work pipeline
- Divide et impera: I will start off with splitting the structure.
  - For every part of Villa Badoer a .csv and a .lines will be produced (https://inkscape.org)
  - Alternatively, if pre-existing pyplasm model for polyhedra are sufficient, then those will be used
- Given a .lines file for a certain part of the structure, produce a graph (V,EV) as lar model
- Given a part of the villa, draw its plant
- Given a plant use cartesian product to build walls etc.
- Figure out a way to use symmetries

# Basement
It is the base on which the main structure lies. It is composed of two layers: basement1 and basement2.
Basement2 is placed upon basement2

In [8]:
VBase1,EVBase1 = larModel(os.getcwd() + "/data/basement1.lines")

In [9]:
VBase1 = [[90.32, 72.02],
 [69.32, 72.08],
 [69.32, 72.54],
 [65.88, 72.54],
 [65.88, 76.21],
 [63.22, 76.21],
 [63.22, 81.31],
 [54.7, 81.31],
 [54.7, 83.39],
 [51.28, 83.41],
 [54.7, 83.41],
 [51.26, 120.98],
 [54.7, 120.98],
 [54.67, 115.42],
 [54.67, 120.98],
 [57.01, 115.42],
 [57.01, 120.98],
 [102.27, 120.98],
 [102.27, 115.43],
 [104.42, 115.42],
 [102.27, 115.42],
 [104.42, 120.97],
 [108.07, 120.97],
 [108.07, 83.41],
 [104.62, 83.41],
 [96.2, 81.31],
 [104.58, 81.31],
 [96.19, 81.29],
 [96.16, 76.24],
 [93.33, 76.21],
 [96.16, 76.21],
 [93.33, 72.54],
 [90.32, 72.54]]

In [11]:
EVBase1 = [[1, 2],
 [2, 3],
 [3, 4],
 [5, 4],
 [6, 5],
 [6, 7],
 [7, 8],
 [8, 9],
 [10, 11],
 [12, 10],
 [13, 12],
 [14, 15],
 [14, 16],
 [17, 16],
 [18, 17],
 [18, 19],
 [20, 21],
 [22, 20],
 [22, 23],
 [23, 24],
 [25, 24],
 [26, 27],
 [25, 27],
 [28, 29],
 [30, 31],
 [32, 30],
 [1, 33],
 [33, 32]]

# Drawing the wire-frame plant

Again from workshop_05

In [13]:
def drawPlant(verts, edges):
    """
    drawPlant(V,EV)
    :param verts: list of vertices in form [X, Y]
    :param edges: list of edges in form [indexofVertA, indexofVertB]
    :return: array of POLILINEs
    """
    plant = []
    for e in edges:
        plant.append(POLYLINE([verts[e[0]-1], verts[e[1]-1]]))
    return plant

In [14]:
plantBase1 = drawPlant(VBase1, EVBase1)
VIEW(STRUCT(plantBase1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x114b15c30> >